In [114]:
import numpy as np
import ast
import pytz
import math
import itertools
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_selection import r_regression 
np.random.seed(1337)
df = pd.read_csv('top10K-spotify.csv',index_col=0)
df["type"].value_counts()
df=df.drop(columns=["index","city_1","district_1","district_2","city_3","district_3" ])
df = df[df['city_2'].notna()]
df = df[df['country'].notna()]
df.rename(columns = {'artist':'name'}, inplace = True)

df2=pd.read_csv('data_w_spotify.csv',index_col=None)
df2=df2.drop(columns=["isdone","id"])
df2['genres'] = df2['genres'].fillna('')

df3=df.merge(df2,on='name')

df3['genres']= df3['genres'].str.strip('[]').str.replace("'","").str.split('\s*,\s*')

def create_soup(x):
    return ' '.join(x['genres']) + ' '+ x['city_2'] + ' ' + x['country']+' ' + x['gender']+ ' ' + x['type']

df3['soup'] = df3.apply(create_soup, axis=1)
print(df3.shape,df3.columns)

dict_names = dict(zip(df3.name, df3.spotifyid))

df = df.reset_index()
indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()



def get_recommendations(name,sim,reverse):
    print(sim[0:20])
    # print(name,sim)
    indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

     # Get the movie indices
    indices = [i[0] for i in sim_scores]
    scores = [round(i[1]*100, 2) for i in sim_scores]

    # Return the top 10 most similar movies
     
    top_10=df3['name'].iloc[indices]
    spot_top10=[]
    for item in top_10:
        spot_top10.append(dict_names[str(item)])

    artist=dict_names[str(df3['name'].iloc[idx])]
    
    wantedDict = {
        'mainId': artist,
        'top10': spot_top10,
        'scores': scores
    }
    return wantedDict




(1416, 11) Index(['name', 'gender', 'age', 'type', 'country', 'city_2', 'spotifyid',
       'genres', 'popularity', 'followers', 'soup'],
      dtype='object')


In [85]:
def CVectorizer():
    count = CountVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def CosSim(x):
    return (cosine_similarity(x,x),True)

In [63]:
def TfidVectorizer():
    count = TfidfVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def EuDSim(x):
    return (euclidean_distances(x,), False)

In [115]:
#matrix=TfidVectorizer()
matrix=CVectorizer()
sim,reverse = EuDSim(matrix)
#im,reverse = CosSim(matrix)

get_recommendations("Coldplay",sim,reverse)

[[0.         4.79583152 5.         ... 5.83095189 5.29150262 5.65685425]
 [4.79583152 0.         4.47213595 ... 5.19615242 4.35889894 5.19615242]
 [5.         4.47213595 0.         ... 4.12310563 3.60555128 3.60555128]
 ...
 [5.56776436 5.29150262 2.44948974 ... 4.58257569 4.58257569 4.12310563]
 [4.89897949 2.64575131 4.12310563 ... 4.89897949 4.         4.89897949]
 [5.74456265 5.29150262 3.46410162 ... 4.79583152 4.79583152 4.35889894]]


{'mainId': '4gzpq5DPGxSnKTe4SA8HAU',
 'top10': ['4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '0sYpJ0nCC8AlDrZFeAA7ub',
  '0sYpJ0nCC8AlDrZFeAA7ub',
  '46CitWgnWrvF9t70C2p1Me',
  '0TytASn0kOZkCpjYTrXsBt',
  '7oPftvlwr6VrsViSDV7fJY'],
 'scores': [173.21,
  173.21,
  173.21,
  173.21,
  173.21,
  200.0,
  223.61,
  223.61,
  244.95,
  244.95]}

In [97]:
def Random_tester():
    rand= np.random.randint(0,len(df3),300)
    return rand

sample=Random_tester()
df_sample=pd.DataFrame(sample)
# df_sample.to_csv("random.csv",index=False)

      0
0   683
1  1341
2   507
3   656
4   796
[ 683 1341  507  656  796  898  633  647  211 1404 1219  973 1137 1023
 1288 1185  507  941 1134  162  482 1217 1227  661 1413  726  405 1180
  626  659  976  522  649 1065  215  664 1366  422  940 1007 1229 1401
  320 1309 1147   93  802  187 1072  556  796  666  638  835  104 1367
  348 1176 1189  102    5  692  317 1309  793  589  454  901   48  627
  102  477  761  764  387  884  806  838  839  870 1085  434  623  452
 1111 1167  494  910  609 1234  639  960 1143  276  804  141 1070 1137
  503 1155  495  907  706 1254  618  521  658  908  846  249    8  917
  609 1004  975 1105  604 1392 1218  474 1144   81  646  289  474  757
 1189  246 1085 1139 1271 1240  254  150 1080    0  625  220  245  380
 1087  219  277  552  998  214  987   59  138    0  169 1202 1360  741
 1076 1127 1408  306  766  405  382  254 1065  698  825   18  943  733
  176 1138 1058  374  518 1385  656  315  531  899 1107  160 1284  578
  680  788  786 1149 1149 136

In [118]:
import requests


apiKey = 'BQCatyFVkL-q4U_32jV-l10mnQvjUqBpUNanWUCOieED3adQvSrNZ2pXn211q8iTim7LtI07ZjkdIhwzs_ShhBpHPXgQydq9ITQ2qxJQUupRNI2EV-wut4H4GRfR_ocRId_HyAUV95DaFveL-m_97vVWWe-Eqh1DbKHhjIWA8PJLlHnYmrSRYOhq-Qfgm_0'
BASE_URL = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {token}'.format(token=apiKey)
}


rand_df=pd.read_csv("random.csv",index_col=None)
print(rand_df.head())
for item,row in rand_df.iterrows():
    id=str(df3['spotifyid'].iloc[row["Num"]])
    
    


    Num
0   683
1  1341
2   507
3   656
4   796
1MXZ0hsGic96dWRDKwAwdr
5UhhpJRFc8vblKlFg9OYTy
3w4VAlllkAWI6m0AV0Gn6a
04abdnqPQe2N4fjztDea6z
1PTl9q5EaEZejVGts7MBLN
4sIl4BTo9l9KqEi0Y3RE72
4X0v8sFoDZ6rIfkeOeVm2i
6BrvowZBreEkXzJQMpL174
6KZDXtSj0SzGOV705nNeh3
44TGR1CzjKBxSHsSEy7bi9
1mbgj8ERPs8lWi7t5cYrdy
3txlfIcKCNrKk5bJw1er3R
1mfDfLsMxYcOOZkzBxvSVW
06nevPmNVfWUXyZkccahL8
57zZoaRDFEjqRwq6FlT0va
1LzWWI9v4UKdbBgz8fqi15
3w4VAlllkAWI6m0AV0Gn6a
4zCH9qm4R2DADamUHMCa6O
238y1dKPtMeFEpX3Y6H1Vr
7CUFPNi1TU8RowpnFRSsZV
1VBflYyxBhnDc9uVib98rw
6p5JxpTc7USNnBnLzctyd4
6vbY3hOaCAhC7VjucswgdS
4F84IBURUo98rz4r61KF70
3W9UldYu0xJcaOAw2SUTDI
1a71mkqZC54WhePSbnq7RK
3tJoFztHeIJkJWMrx0td2f
5DE0by3fZT0nrlJSutY2D7
1OBizG9nrnHJkrL9NjnWYN
3gkbRDj7pkIylWVBpRfrQf
5jcswtptZttxBwWKXbR5k5
7o9Nl7K1Al6NNAHX6jn6iG
2qk9voo8llSGYcZ6xrBzKx
0XgxgYnzpP0KwA4BMliFlq
4kI8Ie27vjvonwaB2ePh8T
0p4nmQO2msCgU4IF37Wi3j
7gbmX8SsfjEjxDMzBi1ZOL
5n9bMYfz9qss2VOW89EVs2
4SdIXLzfabqU61iK7SnKAU
3AuMNF8rQAKOzjYppFNAoB
5iWNGhdgW9MA0l5AqFLqh9
4nrhMdsYS